<a href="https://colab.research.google.com/github/xalejandrow/interacting-with-the-twitter-api-project-tutorial/blob/main/Crimes_4GKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 1. Read the dataset
url = 'https://datos.cdmx.gob.mx/dataset/12d22477-bcf1-49ee-92aa-16a0d0a5817c/' + \
      'resource/3f308147-b1fc-49a9-92b7-e74f3f79aa9c/download/' + \
      'carpetas-de-investigacion-pgj-cdmx.csv'
df = pd.read_csv(url)

In [ ]:
# 2. View the basic data structure and dimensions 

In [ ]:
# 3. Display some observations

In [ ]:
# We want only the mayors from Ciudad de Mexico


In [ ]:
# Convert the values to the right type

In [ ]:
# Summarise the data, including dates

In [ ]:
# How many crimes are recorded in this dataset? In which period?


In [ ]:
# How many crimes were recorded by year?

In [ ]:
# Show the top 5 crimes

In [ ]:
# Calculate the accumulated rate of the top 5 crimes

In [ ]:
# Which city has the most and the less crimes in the dataset?

In [ ]:
# In each city, what are the top 3 subcities ("Colonias") with most crimes?

In [ ]:
# Do you see a trend per month or day?